In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense, Conv3D, MaxPooling3D, Dropout, LSTM, Input, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.regularizers import l2
import numpy as np

In [ ]:
# Input shapes
RNN_in = Input(shape=(30, 123))

# LSTM
RNN1 = LSTM(units=128, return_sequences=True)(RNN_in)
RNN2 = LSTM(units=64, return_sequences=True, dropout = 0.3)(RNN1)

# Flatten
F2 = Flatten()(RNN2)

# Dense layers
DS1 = Dense(64, activation='relu', kernel_initializer='he_normal')(F2)
output = Dense(10, activation='softmax')(DS1)

# Create the model
model = Model(inputs= RNN_in, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 123)]         0         
                                                                 
 lstm (LSTM)                 (None, 30, 128)           129024    
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            49408     
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 64)                122944    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 302026 (1.15 MB)
Trainable params: 302026 (1.15

## Load Npz

In [ ]:
import numpy as np

# Load the keypoints data from the npz file
data1 = np.load('/content/drive/MyDrive/lsa64/Npzs/keypoints_data_i2_j4_k3.npz')

labels = data1['labels']
keypoints = data1['keypoints']


print("Label:")
print(type(labels.item()))

print("Keypoints:")
print(keypoints.shape)

## Padding Key points

In [ ]:
import numpy as np

def pad_or_truncate_keypoints(keypoints, target_frames=30):
    current_frames = len(keypoints)

    if current_frames < target_frames:
      # If there are fewer frames, pad the keypoints with the last frame
      padding = target_frames - current_frames
      pad_value = np.expand_dims(keypoints[-1], axis=0)  # Take the last frame to pad
      pad_values = np.tile(pad_value, (padding, 1, 1))
      keypoints = np.concatenate((keypoints, pad_values), axis=0)
    elif current_frames > target_frames:
        excess_frames = current_frames - target_frames
        front_truncate = excess_frames // 2
        back_truncate = excess_frames - front_truncate
        keypoints = keypoints[front_truncate:-back_truncate]
    return keypoints

## Early Stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=7,verbose=1,
    restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=4,min_lr=1e-7,verbose=1)

callbacks = [early_stopping,lr_scheduler]

In [ ]:
keypoints_folder = '/content/drive/MyDrive/lsa64/Npzs'
batch_size = 3

In [ ]:
def load_keypoints_data(label, person, selected_videos):
    keypoints_data = []
    labels_data = []
    for k in selected_videos:
        filename = f'keypoints_data_i{label}_j{person}_k{k}.npz'
        file_path = os.path.join(keypoints_folder, filename)
        data = np.load(file_path)
        label = int(data['labels'])
        labels_data.append(label)
        keypoints_data.append(pad_or_truncate_keypoints(data['keypoints']).reshape(30,123))
    return np.stack(keypoints_data,  axis=0), labels_data

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np

def train_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [1, 2, 3]  # Specify the selected videos (k values)
    batch_size = 3

    while True:
        for label in range(1, num_labels+1):
            for person in range(1, num_persons + 1):
                keypoints_batch, labels_batch = load_keypoints_data(label, person, selected_videos)
                #pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        #pixel_batch[start:end],
                        keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del keypoints_batch, one_hot_labels


In [ ]:
def val_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [4]  # Specify the selected videos (k values)
    batch_size = 1

    while True:
        for label in range(1, num_labels+1):
            for person in range(1, num_persons + 1):
                keypoints_batch,labels_batch = load_keypoints_data(label, person, selected_videos)
                #pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        #pixel_batch[start:end],
                        keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del keypoints_batch, one_hot_labels

In [ ]:
import os
train_generator = train_batch_generator()
val_generator = val_batch_generator()

history = model.fit(train_generator, batch_size = 3,steps_per_epoch = 11, epochs = 20, validation_data = val_generator
                    ,validation_steps = 11, validation_batch_size=1, callbacks=callbacks)

Epoch 1/20
11/11 [==============================] - 0s 40ms/step - loss: 0.8218 - accuracy: 0.7273 - val_loss: 0.5439 - val_accuracy: 0.9091 - lr: 5.0000e-04
Epoch 2/20
11/11 [==============================] - 0s 36ms/step - loss: 3.6177 - accuracy: 0.2424 - val_loss: 3.3952 - val_accuracy: 0.5455 - lr: 5.0000e-04
Epoch 3/20
11/11 [==============================] - 0s 23ms/step - loss: 2.9005 - accuracy: 0.2727 - val_loss: 0.7408 - val_accuracy: 0.6364 - lr: 5.0000e-04
Epoch 4/20
11/11 [==============================] - 0s 21ms/step - loss: 0.7947 - accuracy: 0.8182 - val_loss: 0.8048 - val_accuracy: 0.7273 - lr: 5.0000e-04
Epoch 5/20
 9/11 [=======================>......] - ETA: 0s - loss: 1.0186 - accuracy: 0.6667
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
11/11 [==============================] - 0s 22ms/step - loss: 0.9601 - accuracy: 0.6667 - val_loss: 2.7287 - val_accuracy: 0.3636 - lr: 5.0000e-04
Epoch 6/20
11/11 [==============================] -

In [ ]:
def test_batch_generator():
    num_labels = 10
    num_persons = 10
    selected_videos = [5]  # Specify the selected videos (k values)
    batch_size = 1

    while True:
        for label in range(1, num_labels + 1):
            for person in range(1, num_persons + 1):
                keypoints_batch,labels_batch = load_keypoints_data(label, person, selected_videos)
                #pixel_batch, labels_batch = load_pixel_data(label, person, selected_videos)
                num_videos = len(selected_videos)
                num_batches = num_videos // batch_size

                for batch_num in range(num_batches):
                    start = batch_num * batch_size
                    end = (batch_num + 1) * batch_size
                    labels_array = np.array(labels_batch) - 1
                    one_hot_labels = to_categorical(labels_array, num_labels)
                    yield (
                        #pixel_batch[start:end],
                        keypoints_batch[start:end]
                    ), one_hot_labels[start:end]
                    del keypoints_batch, one_hot_labels



In [ ]:
test_generator = test_batch_generator()
test_results = model.evaluate(test_generator, steps=11)

11/11 [==============================] - 0s 16ms/step - loss: 0.5501 - accuracy: 0.9091


In [ ]:
test_results

[0.5501289963722229, 0.9090909361839294]